# Intermediate Python - Week 6-8

During the next three weeks, we'll be doing a mini-project to introduce you to statistics and machine learning! \
You'll be making use of lots of the packages we've introduced you to already, so refer back to notebooks \
from previous weeks for a refresher.


